# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!pip install 

ERROR: You must give at least one requirement to install (see "pip help install")


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
 cities_df = pd.read_csv('output_data/cities.csv')
cities_df.count()

City          534
Cloudiness    534
Country       530
Date          534
Humidity      534
Lat           534
Lng           534
Max Temp      534
Wind Speed    534
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = cities_df["Humidity"]

# Plot Heatmap
fig = gmaps.Map()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Map(configuration={'api_key': None}, data_bounds=[(-45.53741323944851, -179.99999), (85.0, 179.99999)], layers…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
Vcity_df = cities_df.loc[cities_df['Max Temp'] < 80, :]
Vcity_df = Vcity_df.loc[Vcity_df['Max Temp'] > 70, :]
Vcity_df = Vcity_df.loc[Vcity_df['Wind Speed'] < 10, :]
Vcity_df = Vcity_df.loc[Vcity_df['Cloudiness'] == 0, :]
Vcity_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
22,Oussouye,0,SN,1607548883,78,12.48,-16.55,75.20,4.70
219,Quelimane,0,MZ,1607548906,78,-17.88,36.89,78.01,3.36
246,Chui,0,UY,1607548909,69,-33.70,-53.46,72.25,9.80
356,Miracatu,0,BR,1607548922,72,-24.28,-47.46,78.71,4.74
372,Valdivia,0,CL,1607548703,49,-39.81,-73.25,71.60,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = Vcity_df
hotel_df['Hotel Name'] = np.nan
#hotel_df
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

idx = 0
for index, row in hotel_df.iterrows():
    # set up a parameters dictionary
    params = {
        "location": f"{row['Lat']}, {row['Lng']}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json['results']
    if len(results) > 0:
        name = response_json['results'][0]['name']
        hotel_df.iloc[idx, -1] = name
    idx = idx + 1



In [10]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
22,Oussouye,0,SN,1607548883,78,12.48,-16.55,75.20,4.70,Campement ALULUM
219,Quelimane,0,MZ,1607548906,78,-17.88,36.89,78.01,3.36,Hotel Elite
246,Chui,0,UY,1607548909,69,-33.70,-53.46,72.25,9.80,Turis Firper Hotel
356,Miracatu,0,BR,1607548922,72,-24.28,-47.46,78.71,4.74,Hotel Ubirajara
372,Valdivia,0,CL,1607548703,49,-39.81,-73.25,71.60,8.05,Hotel Naguilan
427,Kibala,0,TD,1607548930,73,9.11,18.35,71.60,3.85,Hôtel des Chasses


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Map(configuration={'api_key': None}, data_bounds=[(-55.62418606005134, -179.99999), (85.0, 179.99999)], layers…